# Introduction to Postgres

## RDBMS

**관계형 데이터베이스 관리 시스템(Rrelational Database Management System; RDBMS)**는 다수의 사용자들이 데이터베이스 내의 데이터를 접근할 수 있도록 해주는 소프트웨어 도구의 집합이다. DBMS는 사용자 또는 다른 프로그램의 요구를 처리하고 적절히 응답하여 데이터를 사용할 수 있도록 해준다. 

PosgreSQL은 확장 가능성 및 표준 준수를 강조하는 관계형 데이터베이스 관리 시스템의 하나로 데이터베이스 서버로서 주요 기능은 데이터를 안전하게 저장하고 다른 응용 소스 소프트웨어로부터 요청에 응답할 때 데이터를 반환하는 작업을 수행한다. 소규모 단일 머신 애플리케이션에서부터 수많은 동시 접속자가 있는 대형 인터넷 애플리케이션에 이르기까지 여러 부하를 관리할 수 있다. 

## Postgres in Python Script

psycopg2는 파이썬 스크립트에서 Postgres를 사용하기 위한 패키지이다. sqlite3와 동일하게 conn 인스턴스와 cursor 인스턴스를 생성해 쿼리를 작성한다. 

```Python
import psycopg2
conn = psycopg2.connect("dbname=dq user=dq")
cursor = conn.cursor() 

query = """SELECT * FROM users"""
cursor.execute(query)
user = cursor.fetchall() 

conn.close()
```

외부 데이터를 Postgres 데이터베이스 내부 테이블에 작성하기 위해서 INSERT INTO 절과 지시자(%s)를 사용하면 수월한 작업을 할 수 있다. cursor.execute() 메소드는 두번째 argument로 지시자로 처리하기 위한 데이터 셋을 입력 받는다. 

```Python
import psycopg2
import csv
conn = psycopg2.connect("dbname=dq user=dq")
cursor = conn.cursor() 

with open("user_accounts.csv", "r") as file :
    next(file)
    reader = csv.reader(file)
    for row in reader : 
        cursor.execute("INSERT INTO users VALUES (%s, %s, %s, %s);", (row[0], row[1], row[2], row[3]))
        
conn.commit()
conn.close() 
                      
```

## SQL transcation

**데이터베이스 트랜잭션(Database Trasaction)**은 데이터베이스 관리 시스템 또는 유사한 시스템에서 상호 작용의 단위이다. 여기서 유사한 시스템이란 트랜잭션의 성공과 실패가 분명하고 상호 독립적이며, 일관되고 믿을 수 있는 시스템을 의미한다. 

Postgres는 서버와 연결된 다수의 사용자가 데이터베이스를 동시에 변경하거나 다루기 때문에 문제가 발생할 수 있다. 트랜잭션은 따로따로 실행되거나 실행되지 못했을 때 문제가 발생할 수 있는 모든 쿼리를 트랜잭션 블록에 넣어 한번에 실행시킨다. 따라서 하나의 트랜잭션이 실패하면 모든 트랜잭션이 실패하게 되고 데이터베이스 정보가 업데이트 되지 않는다.

Postgres 내부에는 connect.commit() 메소드를 통해서 트랜잭션의 쿼리를 실행한다. conn 인스턴스가 생성되고 close() 메소드를 통해 닫힐 떄까지 실행 되는 모든 쿼리가 트랜잭션 블록이 된다. 

커밋한 사항을 적용하고 싶지 않으면 connect.rollback() 메소드를 통해 트랜잭션을 제거할 수 있다. 

```Python
import psycopg2 
conn = psycopg2.connect("dbname=dq user=dq")
cursor = conn.cursor()

query_string = """
    CREATE TABLE users(
        id integer PRIMARY KEY,
        email text,
        name text,
        addres text 
    );
"""

cursor.execute(query_string)
conn.commit()
conn.close()
```

# Creating Tables 

## Check Datatype 

적절하지 않은 데이터 타입 스키마는 쿼리문을 작성할 때 오류를 발생시킨다. psycopg2 패키지의 cursor.description 속성은 데이터베이스 테이블의 데이터 타입을 출력한다. curosr.description 속성은 name과 type_code로 이루어진 Column 객체로 구성되어 있다. name은 열의 이름을, type_code는 데이터 타입에 상응하는 정수 값을 가지고 있다. pg_catalog.pg_type 테이블은 type_code(oid)와 데이터 타입 이름(typename)으로 구성되어 있다.


| Name | Type | Description |
|:---:|:---:|:---:|
| oid | oid | Row identifier | 
| typename | name | Data type name | 


```Python
import psycopg2
conn = psycopg2.connect("dbname=dq user=dq")
cursor = conn.cursor()

query = "SELECT * FROM table LIMIT 0;" 
curosr.execute(query)
print(cursor.description)

# The type_code will be in 'nn'

query = "SELECT typename FROM pg_catalog.pg_type WHERE oid=nn;"
cursor.execute(query)
type_of_nn = cursor.fetchone()[0]
print(f"The typename of type code : {type_of_nn}")
```

## Optimizing Numerical variable

데이터베이스 테이블은 데이터를 저장하는 공간으로써 저장 공간이 필요하다. 만약 불필요하게 넓은 크기를 가지고 있는 데이터 타입을 가지고 있다면 공간 낭비와 속도 낭비를 초래한다. 따라서 데이터 타입을 최적화 하는 작업은 반드시 필요하다. 모든 데이터를 오류 없이 출력 가능한 데이터 타입을 선정한 후 ALTER 절을 사용해 데이터 타입을 변경한다. 

```Python
import psycopg2
conn = psycopg2.connect("dbname=dq user=dq")
cursor = conn.cursor()

query = "ALTER TABLE users ALTER COLUMN id TYPE int4;"
cursor.execute(query)

conn.commit()
conn.close()
```

## Optimizing Text variable

텍스트 데이터의 경우 테이블 내부에 어떤 길이의 문자열도 저장 가능하다. 이런 유연성은 데이터를 저장하는데 필요한 공간의 낭비를 불러온다. 따라서 각 열의 고유값에 대해 가장 긴 길이의 문자열을 기준으로 데이터의 크기를 제한하는 작업이 필요하다. 

```Python
import csv 
with open('users.csv', 'r') as file : 
    next(file)
    reader = csv.reader(file)
    unique_words = set()
    for row in reader : 
        check_col = row[n]
        unique_words.add(score_phrase)
        
max_len = 0
for unique_word in unique_words : 
    if len(unique_word) >= max_len : 
        max_len = len(unique_word) 
print(max_len)

import psycopg2
conn = psycopg2.connect("dbname=dq user=dq")
cursor = conn.cursor()

query = "ALTER TABLE user ALTER COLUMN email TYPE varcahr(11);"
cursor.execute(query)

conn.commit()
conn.close() 
```

## Optimizing using enumerated datatype 

모든 고유값을 확인해서 가장 큰 길이의 문자열을 확인하는 작업보다 열거형 데이터 타입을 사용하게 되면 좀 더 효율적인 작업이 간으하다. 열거형 타입은 명명된 값의 집합을 이루는 자료형으로, 해당 언어의 상수 역할을 하는 식별자이다. 열거형 자료는 각각의 색인에 4바이트가 할당되어 pg_enum 테이블에 저장된다. 한번 저장된 열거형 자료는 Postgres 엔진이 값을 바로 확인해서 저장할 수 있도록 한다.

```Python
import psycopg2
conn = psycopg2.connect("dbname=dq user=dq")
cursor = conn.cursor()

query = """
CREATE TYPE enum_name AS ENUM(
    'val1', 'val2', 'val3', ... 
); 

ALTER TABLE table_name 
ALTER COLUMN column TYPE enum_name
USING CAST(col_name AS enum_name);
"""
cursor.excute(query)

conn.commit()
conn.close() 
```

## Create new variable and insert 

```Python
import datetime
import psycopg2
import csv
conn = psycopg2.connect("dbname=dq user=dq")
cur = conn.cursor()

query1 = "ALTER TABLE ign_reviews ADD COLUMN release_date date;"
query2 = """
ALTER TABLE ign_reviews
DROP COLUMN release_year,
DROP COLUMN release_month,
DROP COLUMN release_day;
"""
cur.execute(query1)
cur.execute(query2)

with open('ign.csv', 'r') as file :
    next(file)
    reader = csv.reader(file)
    for row in reader : 
        year = int(row[8])
        month = int(row[9])
        day = int(row[10])
        date = datetime.date(year, month, day)
        row = row[:-3]
        row.append(date)
        query = "INSERT INTO ign_reviews VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);"
        cur.execute(query, row)

conn.commit()
conn.close()
```

# Prepared Statement and SQL injection

## SQL injection

SQL injection은 응용 프로그램 보안 상의 허점을 의도적으로 이용해, 악의적인 SQL 문을 실행되게 함으로써 데이터베이스를 비정상적으로 조작하는 코드 인젝션 공격 방법이다. 단순히 값을 받아서 수행하는 쿼리문에 "admin OR 1=1;--"를 입력하면 쿼리문은 모든 행을 출력하게 된다. 

```Python
query = "SELECT * FROM users WHERE username = '" + name + "';"

# if name = "admin OR 1=1;--"
query = "SELECT * FROM users WHERE username = admin OR 1=1;"
```

## %s Placeholder

Postgres의 cursor.execute() 메소드는 자리 표시자(placholder)를 입력받아 데이터베이스를 조작할 수 있다. 함수 내부에 입력 가능한 자리 표시자는 두가지가 있다.

- %s : 순서에 따라 값을 대체하는 자리 표시자
- %(key)s : key값에 따라 값을 대체하는 자리 표시자 

%(key)s 방법의 경우 튜플, 리스트 대신에 딕셔너리 형태로 존재하는 key : value를 입력해야 한다.

```Python
import psycpog2
conn = psycopg2.connect("dbname=dq user=dq")
cursor = conn.cursor()

query = """INSERT INTO users VALUES (%s, %s, %s, ..., %s);"""
cursor.execute(qeury, ('val1', 'val2', ... 'valn'))

conn.commit()
conn.close()
```

```Python
import psycpog2
conn = psycopg2.connect("dbname=dq user=dq")
cursor = conn.cursor()

row_dict = {'key1' : 'val1' : 'key2' : 'val2', ... , 'keyn' : 'valn'}
query = """INSERT INTO users VALUES (%(key1)s, %(key2)s, ..., %(keyn)s);"""
cursor.execute(qeury, row_dict)

conn.commit()
conn.close()
```

## Prepared Statement 

Prepared Statement는 추가적으로 뒤따라오는 SQL 문을 무시하고 지정되어 있는 형태의 쿼리를 수행할 수 있도록 만든다. 즉 Prepared Statement는 SQL injection을 방어하기 위해 미리 지정된 함수와 같은 개념으로 사용된다. 지정된 Prepared Statement는 EXCUTE 절을 사용해 실행한다. 

```Python
import psycopg2
conn = psycopg2.connect("dbname=dq user=dq")
cursor = conn.cursor()

cursor.execute("""
    PREPARE pre_name(int4, text, text, ... ) AS 
        INSERT INTO users VALUES ($1, $2, $3, ..., $n)
""")

cursor.execute("EXECUTE pre_name(%s, %s, ..., %s)", ('val1', 'val2', ..., 'valn'))

conn.commit()
conn.close()
```

# Loading and Extracting Data with Tables 

## What is mogrifying 

Python script에서 Postgres 데이터베이스를 다룰때 외부의 값을 사용하려면 placeholder를 사용해 튜플을 추가적으로 입력해야한다. cursor.execute() 메소드에 전달된 값은 자동으로 Postgres 내부에서 읽을 수 있는 값으로 변환되어 전송되어야 한다. curor.mogrify() 메소드는 Postgres 엔진이 읽을 수 있는 바이트 객체로 데이터를 변환해준다. 

```Python
mogrified_data = cur.mogrify("(%s, ..., %s)", tuples)

# decode mogrified_data
mogrified_string = mogrified_data.decode(conn.encoding)
```

## Loading data with cursor.mogrify()

외부 데이터를 Postgres 데이터베이스 내부에 입력하기 위해서는 지시자를 통해 튜플 값을 cursor.execute()에 넣고, cursor.execute() 내부에서 Postgres 엔진이 읽을 수 있는 값으로 mogrify 한 뒤에 값을 추출한다. cursor.mogrify()를 진행한뒤 일반적인 string 형식으로 쿼리문을 작성하게 되면 한번에 많은 양의 반복 쿼리를 빠르게 수행할 수 있다. 

cursor.mogrify() 메소드를 사용하고 디코딩 된 데이터들은 placeholder를 통해 정의되어 있기때문에 SQL injection 문제에 대해서도 어느정도 자유롭다. 

```Python
import csv
import psycopg2

conn = psycopg2.connect("dbname=dq user=dq")
cur = conn.cursor()

with open("ign.csv", mode = 'r') as file : 
    next(file)
    reader = csv.reader(file) 
    rows = [row for row in reader]
    
    mogrified_rows = [cur.mogrify("(%s, %s, %s, %s, %s, %s, %s, %s, %s)", row) for row in rows]
    decoded_rows = [row.decode(conn.encoding) for row in mogrified_rows]
    insert_string = ",".join(decoded_rows)
    cur.execute("INSERT INTO ign_reviews VALUES " + insert_string + ";")
    conn.commit()
    conn.close()
```

## Loading data with cursor.copy_from() 

cursor.mogrify()는 지시자를 사용하는 방식보다는 빠르긴 하지만 결국 각 행을 하나씩 읽어들이고 디코딩 한 이후 join 연산을 해서 쿼리문을 수행해야 한다. cursor.copy_from() 메소드는 모든 행을 복사하는 경우에 대해서 cursor.mogrify() 메소드 보다 빠르게 전체 데이터를 옮길 수 있다. 


```Python
import psycopg2

conn = psycopg2.connect("dbname=dq user=dq")
cur = conn.cursor()

with open("ign.csv", mode = 'r') as file : 
    next(file) 
    cur.copy_from(file, "ign_reviews", sep = ',')
```

하지만 cur.copy_from() 메소드는 csv 파일을 읽을 때, 각 데이터가 ','가 있을 경우 분할된 열로 인식하기 때문에 오류가 발생할 수 있다. cur.copy_export() 메소드는 세부적인 옵션을 작성함으로써 데이터 로딩 작업을 수월하게 할 수 있다. 데이터를 테이블로 옮기기 위해서는 COPY ~ FROM STDIN 쿼리문을 사용하면 된다. 

```Python
import csv
import psycopg2
conn = psycopg2.connect("dbname=dq user=dq")
cursor = conn.cursor()

with open('ign.csv', mode = 'r') as file :
    cursor.copy_expert("COPY ign_reviews FROM STDIN CSV WITH HEADER", file)
conn.commit()
conn.close()
```

## Fastest way to loading data

파이썬 스크립트에서 Postgres DB에 데이터를 업로딩한다고 했을때 가능한 방법은 3가지가 있다.

1. 지시자를 활용하여 각각의 행을 개행으로 입력하는 방법
2. cursor.mogrify()를 이용해 모든 행을 한번에 업로드 하는 방법
3. cursor.copy_expert()를 이용해 데이터 전체를 한번에 테이블로 업로드 하는 방법

timeit 패키지의 timetit() 함수를 활용해서 각각의 방법의 시간을 측정한 결과 3, 2, 1 순서로 속도가 빨랐다. 

```Python
import csv
import psycopg2

conn = psycopg2.connect("dbname=dq user=dq")
cur = conn.cursor()

# Multiple single insert statements
def multiple_inserts():
    with open("ign.csv", "r") as f:
        next(f)
        reader = csv.reader(f)
        for row in reader:
            cur.execute("INSERT INTO ign_reviews VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);", row)
    conn.rollback()
        
# Multiple mogrify insert
def mogrified_insert():
    with open("ign.csv", "r") as f:
        next(f)
        reader = csv.reader(f)
        mogrified = [ 
            cur.mogrify("(%s, %s, %s, %s, %s, %s, %s, %s, %s)", row).decode(conn.encoding)
            for row in reader
        ] 
        mogrified_values = ",".join(mogrified) 
        cur.execute("INSERT INTO ign_reviews VALUES " + mogrified_values + ";")
    conn.rollback()
    
# Copy expert method
def copy_expert():
    with open("ign.csv", "r") as f:
        cur.copy_expert("COPY ign_reviews FROM STDIN WITH CSV HEADER;", f)
    conn.rollback()

# Add your code under
import timeit

time_multiple_inserts = timeit.timeit(multiple_inserts, number = 1)
time_mogrified_insert = timeit.timeit(mogrified_insert, number = 1)
time_copy_expert = timeit.timeit(copy_expert, number = 1)

print(f"Consumed times using multiple insert : {time_multiple_inserts}")
print(f"Consumed times using mogrified insert : {time_mogrified_insert}")
print(f"Consumed times using copy expert : {time_copy_expert}")
```

## Extracting data

cursor.copy_expert() 메소드는 데이터를 업로딩 하는것 뿐만 아니라 테이블의 데이터를 파일로 추출할 수 있다. 이때 FROM을 TO, STDIN을 STDOUT으로 변경하고 파일을 오픈할 때 'w' 모드로 작성해야 한다. 

```Python
import psycopg2
conn = psycopg2.connect("dbname=dq user=dq")
cursor = conn.cursor()

with open("ign_copy.csv", "w") as file : 
    cursor.copy_expert("COPY ign_reviews TO STDOUT WITH CSV HEADER;", file)
    conn.close()
```

## Transferring data between tables 

두 테이블 간 데이터를 전송하는 방법은 두가지가 있다. 첫번째는 기존의 테이블의 데이터를 보존할 temporary file을 생성하고 cursor.copy_expert("COPY ... TO STDOUT ...")으로 데이터를 외부 저장소에 저장한 뒤 cursor.copy_expert("COPY ... FROM STDIN ...") 메소드로 테이블에 옵로딩 하는 방법이 있다. 하지만 해당 방법은 데이터의 행이 수백만개가 넘을경우 시간, 공간상 효율 문제가 발생한다. 두번째 방법은 SQL 내부에서 테이블 간 데이터를 전송하는 방법이다. INSERT INTO 쿼리문에서 VALUES를 서브쿼리로 작성해서 전송하고자 하는 테이블의 데이터를 전송할 수 있다. 

```Python
import psycopg2
# the query for you to create the empty table copy
create_string = """
CREATE TABLE ign_reviews_copy (
    id bigint PRIMARY KEY,
    score_phrase evaluation_enum,
    title varchar(200),
    url varchar(200),
    platform platform_enum,
    score decimal(3, 1),
    genre genre_enum,
    editors_choice boolean,
    release_date date
);
"""

conn = psycopg2.connect("dbname=dq user=dq")
cursor = conn.cursor() 

with open('temp.csv', 'w') as file :
    cursor.copy_expert("COPY ign_reviews TO STDOUT WITH CSV HEADER;", file)
cursor.execute(create_string)
with open('temp.csv', 'r') as file :
    cursor.copy_expert("COPY ign_reviews_copy FROM STDIN WITH CSV HEADER;", file)
conn.commit()
conn.close()
```

```Python
import psycopg2
conn = psycopg2.connect("dbname=dq user=dq")
cursor = conn.cursor()

cursor.execute("""CREATE TABLE ign_restricted (
    id bigint PRIMARY KEY,
    title varchar(200),
    release_date date
);""")
cursor.execute("""
    INSERT INTO ign_restricted (id, title, release_date) 
    	SELECT id, title, release_date FROM ign_reviews;
""")

conn.commit()
```

# Users and Database Management 

## Connection to Postgres

SQLite3와 다르게 Postgres는 클라이언트-서버 모델이기 때문에 각기 다른 유저들이 서로 다른 권한을 가지고 데이터베이스에 접속한다. 유저의 보안성을 위해 Postgres는 password를 설정하여 데이터베이스 서버에 접속할 수 있다. **슈퍼유저(Superuser)**는 파일 시스템의 sudo 권한과 비슷하게 Postgres 서버에 완전한 권한을 가지고 있으면서 다른 유저들의 권한을 조정할 수 있다.

```Python
import psycopg2
conn = psycopg2.connect(dbname="dq", user="postgres", password="abc123")
```

## Create user

슈퍼유저로 데이터베이스에 접속한 경우 새로운 유저를 옵션을 부여하여 생성할 수 있다. 생성문에 WITH 옵션을 붙여 유저의 권한을 부여하며 NO를 옵션앞에 붙여 제한할 수 있다.

```Python
import psycopg2
conn = psycopg2.connect(dbname="dq", user="postgres", password="abc123")
cursor = conn.connect()

cursor.execute("CREATE USER data_viewer WITH SUPERUSER PASSWORD 'secret';")
conn.commit()
conn.close()
```

## Change user 

슈퍼유저의 경우 User privileges를 무시하기 떄문에 신중하게 설정해야한다. 유저의 옵션을 변경하고 싶은 경우 ALTER USER ~ 를 사용한다. 

```Python
impory psycopg2
conn = psycopg2.connect(dbname="dq", user="postgres", password="abc123")
cursor = conn.connect()

cursor.execute("ALTER USER data_viewer WITH NOSUPERUSER;")
conn.commit()
conn.close()
```

## Check users 

SQL은 모든 정보를 table로 저장한다. 유저의 정보도 pg_table 내부에 저장되어 있다. 

```Python
import psycopg2
conn = psycopg2.connect(dbname="dq", user="postgres", password="abc123")
cursor = conn.cursor()

cursor.execute("SELECT * FROM pg_user;")
users = cursor.fetchall()

for row in users : 
    print(row)
conn.close() 
```

## User Privileges 

유저를 생성하고 가장 먼저 해야할 일은 유저의 password를 생성하는 것이다. 이후 유저에게 사용가능한 명령어의 범위를 지정해준다. **User Privileges**는 유저가 사용할 수 있는 명령어의 집합으로. 대부분 DROP, DELETE, UPDATE와 같이 DDL 언어를 제한한다. 유저에게 부여하는 권한은 반드시 최소 명령어만 제공해야하며 이를 **Least Privilege Principle**이라고 한다. 즉, 먼저 REVOKE 명령어를 통해 유저의 모든 권한을 박탈하고, GRANT 명령어를 통해 부여할 명령어를 입력하는 방식으로 진행한다.

권한을 부여하거나 뺏을 때 명령어와 작업가능한 테이블을 명시해줘야 한다. 

- REVOKE/GRANT [commands] ON [table] FROM/TO [user]

```Python
import psycopg2
conn = psycopg2.connect(dbname="dq", user="postgres", password="abc123")
cursor = conn.cursor()

cursor.execute("REVOKE ALL ON users FROM data_viewer;")
cursor.execute("GRANT SELECT ON users TO data_viewer;") 
conn.commit()
conn.close()
```

## Check Users' Privileges on table 

REVOKE와 GRANT 명령어를 통해 부여한 유저의 권한을 확인해야 하는 경우가 있다. information_schema.table_privileges는 테이블과 유저에 따라 부여된 특권을 확인할 수 있다.

테이블 내부에는 다음과 같은 정보가 존재한다.

- grantor : privilege를 허용한 아이디(주로 슈퍼유저)
- grantee : privilege를 받은 아이디
- table_catalog : 테이블을 보유하고 있는 데이터베이스
- table_name : 테이블 명 
- privilege_type : 부여받은 권한 집합 

```Python
import psycopg2
conn = psycopg2.connect(dbname="dq", user="postgres", password="abc123")
cursor = conn.cursor()

cursor.execute("SELECT grantor, grantee, privilege_type FROM information_schema.table_privileges WHERE table_name = 'users';")
privileges = cursor.fetchall()

for row in privileges 
    print(row)
conn.close()
```

## Create Group 

User privileges 중 SELECT 만 허용되는 유저를 **readonly user**라고 한다. 하지만 유저의 수가 매우 많아지고 readonly user를 일일이 지정하는 것은 매우 번거롭다. Postgres는 GROUP 명령어를 통해 동일한 user privileges를 가지고 있는 유저를 하나의 그룹으로 묶어 관리할 수 있다. CREATE GROUP ~ 으로 그룹명을 생성한다.(이때 사용되는 NOLOGIN 옵션은 유저가 개인 아이디로 로그인 할 수 있도록 정해주는 작업이다.) 이후 생성된 그룹은 GRANT/REVOKE를 통해 그룹이 가능한 privileges를 정의하고, 유저를 그룹에 넣어주면 된다.

```Python
import psycopg2
conn = psycopg2.connect(dbname="dq", user="postgres", password="abc123")
cursor = conn.cursor()

cursor.execute("CREATE GROUP readonly NOLOGIN;")
cursor.execute("REVOKE ALL ON users FROM readonly;")
cursor.execute("GRANT SELECT ON users TO readonly;")
cursor.execute("GRANT readonly TO data_viewer;") 
conn.commit()
conn.close()
```

## Create Database 

Postgres 서버 내부에는 복수의 데이터베이스가 존재할 수 있다. 데이터베이스는 CREATE 구문을 사용해서 상성 가능하며 데이터베이스 오너를 지정 가능하다.

```Python
import psycopg2
conn = psycopg2.connect(dbname="dq", user="dq")
conn.autocommit = True 
cursor = conn.cursor()

cursor.execute("CREATE DATABASE my_database OWNER postgres;")
conn.autocommit = False 
```

## Database Previeges

데이터베이스 또한 데이터베이스에 어떤 유저가 접근 가능한지에 대해 정의할 수 있다. Postgres에서 전체 유저의 그룹을 'public'이라고한다. 따라서 데이터베이스의 previleges를 제한할 때 먼저 public 그룹에 대해 전체 previleges를 제한한 후 원하는 그룹의 previleges를 제한해야 한다. 그렇지 않을경우 public 그룹에 존재하는 previleges가 하위 그룹의 previleges에 상속되는 문제가 발생한다. 

```Python
import psycopg2
conn = psycopg2.connect(dbname="my_database", user="postgres", password="abc123")
cursor = conn.cursor()

cursor.execute("REVOKE ALL ON DATABASE my_database FROM public;")
cursor.execute("GRANT CONNECT ON DATABASE my_database TO readonly;")

conn.commit()
conn.close()

```

## Schema 

**데이터베이스 스키마(Database Schema)**는 데이터베이스의 자료의 구조, 자료의 표현 방법, 자료간의 관계를 형식언어로 정의한 구조이다. 데이터베이스 관리 시스템이 주어진 설정에 따라 데이터베이스 스키마를 생성하며, 데이터베이스 사용자가 자료를 저장, 조회, 삭제, 변경할 때 DBMS는 자신이 생성한 데이터베이스 스키마를 참조하여 명령을 수행하게 된다.

스키마는 데이터베이스 내부의 테이블들을 정돈하는 파일첩과 같다. 하나의 데이터베이스는 여러개의 스키마를 가지고 있고, 각각의 스키마는 여러개의 테이블을 보유하고 있다. 스키마 내부의 테이블에 접근할 때 schema_name.table_name 형식을 따르는데 앞서 테이블의 user privileges를 확인하기 위해 information_schema.table_privleges를 참조한 것 과 동일하다.

스키마를 생성하는 작업은 앞서 했던 것과 마찬가지로 CREATE 명령어를 통해 가능하며 각 유저에 스키마에 대한 접근권을 주는 명령어 또한 GRANT/REVOKE를 통해 가능하다.

```Python
import psycopg2
conn = psycopg2.connect(dbname="my_database", user="postgres", password="abc123")
cursor = conn.cursor()

cursor.execute("CREATE SCHEMA my_schema;")
cursor.execute("REVOKE DELETE ON ALL TABLES IN schema_name FORM user_name;")
cursor.execute("GRANT SELECT ON ALL TABLES IN schema_name TO user_name;")
conn.commit()
conn.close()
```

# Exploring Postgres Internals

## Internal tables 

모든 Postgres 엔진 내부에는 데이터베이스 전체 구조를 관리하는데 사용되는 스키마가 있다, 이 스미카는 각각 public, information_schema, 그리고 pg_catalog 스키마라고 불린다. 해당 테이블들은 데이터베이스에 대한 정보, 테이블의 이름, 테이블 내부에 저장되어 있는 데이터 타입에 대해 정의하고 있다. Internal table들을 사용하면 처음보는 데이터베이스의 구조를 파악하는데 도움을 얻을 수 있다. 

- pg_catalog : system catalog tables
- information_schema : information catalog tables 
- public : default schema for user created tables 

유저가 CREATE TABLE 명령문을 통해 생성한 테이블은 public 스키마 내부에 저장된다. 유저에 대한 시스템 정보는 pg_catalog.pg_user에, 테이블에 대한 정보는 pg_catalog.pg_tables에, 그리고 데이터 타입에 대한 정보는 pg_catalog.pg_type에 저장되어 있다. cursor.description은 각 테이블 내부의 데이터에 대한 컬럼명과 데이터 타입 코드를 저장하고 있는 속성이다. 

```Python
import psycopg2
conn = psycopg2.connect("dbname=hud user=hud_admin password=hud_pwd")
cursor = conn.cursor()

cursor.execute("SELECT tablename FROM pg_catalog.pg_tables ORDER BY tablename;")
table_names = cursor.fetchall()

print("Number of tables in database : {}".format(len(table_names)))

for table in table_names :
    print(table)
```

## Check table in public schema 

pg_tables 테이블 내부에는 schemaname과 tablename이 저장되어 있어, public 스키마의 테이블을 가져올 수 있다. 

```Python
import psycopg2
conn = psycopg2.connect("dbname=hud user=hud_admin password=hud_pwd")
cursor = conn.cursor()
cursor.execute("""
    SELECT tablename FROM pg_catalog.pg_tables
    WHERE schemaname NOT IN ('pg_catalog', 'information_schema')
    ORDER BY tablename;
""")

table_names = cursor.fetchall()
print("Number of table user created : {}".format(len(table_names)))

for table in table_names : 
    print(table)
```

## Check typecode of each columns from tables 

public 스키마에서 확인한 둘 이상의 테이블에 대해서 내부 데이터의 타입을 반복적으로 확인하기 위해선 for loop가 필요하다. Postgres는 SQL injection 문제를 방지하기 위해 cursor.execute()의 second positional argument를 사용해야 한다. 하지만 테이블 명은 문자열 데이터로 입력하게 되면 Postgres 엔진이 쿼리문을 읽지 못하는 문제가 발생한다. 

psycopg2.extensions 패키지읜 AsIs 모듈은 입력된 모듈을 Postgres 내부에서 쿼리문 그대로 인식할 수 있게 문자열을 변경하는 역할을 수행한다. 따라서 테이블 이름에 따라 열의 정보를 저장하는 col_descriptions dictionary에 저장이 가능하다.

```Python
import psycopg2
conn = psycopg2.connect("dbname=hud user=hud_admin password=hud_pwd")
cursor = conn.cursor()

from psycopg2.extensions import AsIs
col_descriptions = {}

for table in table_names : 
    cursor.execute("SELECT * FROM %s LIMIT 0;", (AsIs(table[0]),))
    col_descriptions[table] = cursor.description
conn.close()
```

## Store typename from type code using pg_catalog.pg_type

cursor.description은 데이터 타입을 type_code로 반환하기 때문에, 사람이 읽을 수 있는 typename으로의 전환이 필요하다. pg_catalog 스키마의 pg_type은 type_code : typename을 저장하는 테이블이다. 

```Python
import psycopg2
conn = psycopg2.connect("dbname=hud user=hud_admin password=hud_pwd")
cursor = conn.cursor()

cursor.execute("SELECT oid, typname FROM pg_catalog.pg_type;")
type_mappings = {int(oid) : typename for oid, typename in cursor.fetchall()}
print(type_mappings[1082])
```

## Readable descriptions

위의 결과를 종합해 테이블명에 따른 컬럼명(이름, 타입, 데이터 크기)과 행의 크기를 저장하는 dictionary를 생성해서 메타데이터를 저장하면 다음과 같다. 

1. 스키마 이름이 'public'인 스키마 내부에서 테이블 이름을 추출해 table_names에 저장한다. 
2. 테이블에 따라 테이블 이름과 type_code를 저장하고 있는 col_descriptions를 생성한다.
3. type_code : typename으로 매칭되어 있는 type_mappings dictionary를 생성한다.
4. 테이블 이름에 따라 'column' : , 'number of rows' : 로 메인 키를 가지고 있고, 'columns' 키 내부에는 칼럼 이름, 타입, 내부 사이즈를 dictionary 형태로 저장하고 있는 list를 생성한다.

```Python
import psycopg2
conn = psycopg2.connect(dbname = "hud", user="hud_admin", password="hud_pwd")
cursor = conn.cursor() 

cursor.execute("""
    SELECT tablename FROM pg_catalog.pg_tables
    WHERE schemaname NOT IN ('pg_catalog', 'information_schema')
    ORDER BY tablename;
""")
table_names = cursor.fetchall()

from psycopg2.extensions import AsIs
col_descriptions = {}
for table in table_names : 
    cursor.execute("SELECT * FROM %s LIMIT 0;", (AsIs(table[0]),))
    col_descriptions[table] = cursor.description
               
cursor.execute("SELECT oid, typname FROM pg_catalog.pg_type;")
type_mappings = {int(oid) : typename for oid, typename in cursor.fetchall()} 

readable_description = {} 
for table in table_names : 
	cursor.execute("SELECT COUNT(*) FROM %s;", (AsIs([table[0]),))
    readable_description[table] = {
        'columns' : [
            {
                'name' : col.name, 
                'type' : type_mappings[col.type_code],
                'internal_size' : col.internal_size
            } for col in col_descriptions[table]
        ],
        "number_of_rows" : cursor.fetchone()[0]
    }
```

## JSON format 

Dictrionary 형태로 저장된 데이터를 컴퓨터와 인간 모두 읽기 쉽게 정리한 포맷을 JSON format이라고 한다. json.dumps()는 dictionary 데이터를 JSON format으로, json.loads()는 JSON format을 dictionary 데이터로 변환하는 메소드이다. 

```Python
import json

json_description = json.dumps(readable_description, indent = 4)
print(json_description)
```

# Debugging Postgres Queries 

## Execution of queries 

1. **Query parsing** : 쿼리문을 올바른 문법 단의로 분할하여 오류가 없을 경우 query tree로 변경한다 
2. **Query rewrite** : system catalog에서 확인한 특별한 규칙을 query tree에 적용한다(Index) 
3. **Query planning** : Planner/Optimizer가 query tree를 최적화하여 최적 시간의 쿼리를 찾는다
4. **Query execution** : 쿼리를 수행한다.

## EXPLAIN 

EXPLAIN 명령어는 SQL 쿼리문을 입력받아 데이터를 출력하는게 아닌 Postgres 엔진이 쿼리문을 어떤 노드로 수행하는지 표기해주는 명령문이다. 즉, Query planning 단계에서 쿼리를 가장 빠르고 효율적으로 수행하는 단계를 출력한다. 

```Python
query = """
EXPLAIN 
[statement to explain]
"""

# result of EXPLAIN command 
[('Seq Scan on homeless_by_coc  (cost=0.00..2145.29 rows=86529 width=480)',)]
```

## Seq Scan

**Seq Scan** 방식은 테이블을 Full Scan하여 레코드를 읽는 방식으로 인덱스가 존재하지 않거나 인덱스가 존재하더라도 읽어야할 범위가 전체 테이블에 거의 범접할 경우에 선택하는 방식이다. 테이블의 모든 행을 하나씩 읽어들인다. 

Seq Scan을 적용하면 쿼리문의 수행 시간의 쿼리 플랜의 길이와 비례하게 된다. 

## Return result of EXPLAIN in format 

EXPLAIN 명령문을 실행한 결과는 텍스트 형태로 출력된다. 정해진 포멧으로 결과를 출력하려면 EXPLIAN 뒤에 (FORMAT [format]) 옵션을 추가하여 사용한다. 

```Python
import psycopg2
conn = psycopg2.connect(dbname="hud", user="hud_admin", password="hud123")
cur = conn.cursor()

import json
cur.execute("EXPLAIN (FORMAT json) SELECT COUNT(*) FROM homeless_by_coc WHERE year > '2012-01-01';")
query_plan = cur.fetchone()

print(json.dumps(query_plan, indent = 2))
```

## Translation Nested Node 

```Python
# Result of query_plan in json format 
[
  {
    "Plan": {
      "Node Type": "Aggregate",
      "Strategy": "Plain",
      "Partial Mode": "Simple",
      "Parallel Aware": false,
      "Startup Cost": 2433.72,
      "Total Cost": 2433.73,
      "Plan Rows": 1,
      "Plan Width": 8,
      "Plans": [
        {
          "Node Type": "Seq Scan",
          "Parent Relationship": "Outer",
          "Parallel Aware": false,
          "Relation Name": "homeless_by_coc",
          "Alias": "homeless_by_coc",
          "Startup Cost": 0.0,
          "Total Cost": 2361.61,
          "Plan Rows": 28843,
          "Plan Width": 0,
          "Filter": "(year > '2012-01-01'::date)"
        }
      ]
    }
  }
]
```

### Nesting Node 

JSON 포맷으로 작성된 query_plan의 결과는 각각의 단계를 의미하는 "Node Type"로 구성되어 있다. 노드는 내장된 노드가 존재하는데, 상위 노드는 내장된 노드에 의존도를 가지고 부모-자식 관계를 형성한다. 즉, query_plan에 저장되어 있는 "Seq Scan"이 먼저 실행되고 이후에 "Aggregate"노드가 실행된다. 해당 구조를 **query plan tree**라고 한다. 

### Cost 

query_plan에 저장되어있는 "Start up Cost"와 "Total Cost"는 텍스트 표기에 표기되어있는 cost=2433.72...2433.73을 나누어 표기한 키이다. 각각의 값은 실제 시간이 아닌 임의적으로 정해진 시간의 단위로, Startup Cost는 행이 전달되기 전까지 걸리는 시간을, Total Cost는 Startup Cost를 포함한 Node plan이 완료될때까지 걸리는 시간을 의미한다. 

### Run Cost for the sequential scan

Seq Scan의 시간 측정은 다음의 공식을 사용한다.   

$$(cpu\_tuple\_cost + cpu\_operator\_cost) \times N_{tuple} + seq\_page\_cost \times N_{page}$$

- cpu_tuple_cost : 쿼리문을 수행하는 동안 한 행을 처리하는데 추정되는 비용(default = 0.01)
- cpu_operator_cost : 쿼리문을 수행하는 동안 연산자와 함수를 처리하는 비용(default = 0.0025)
- seg_page_cost : 디스크 페이지를 fetching하는 비용(default = 1.0)
- $N_{tuples}$ : 테이블 내부의 튜플의 수
- $N_{page}$ : 테이블에 사용되는 디스크 페이지의 수

```Python
import psycopg2
cpu_tuple_cost = 0.01
cpu_operator_cost = 0.0025
seq_page_cost = 1.0

conn = psycopg2.connect(dbname="hud", user="hud_admin", password="hud123")
cursor = conn.cursor()

cursor.execute("SELECT reltuples, relpages FROM pg_class WHERE relname = 'homeless_by_coc';")

n_tuple, n_page = cursor.fetchone()

total_cost = (cpu_tuple_cost + cpu_operator_cost) * n_tuple + seq_page_cost * n_page 
print(total_cost)
```

## Run cost in Real time 

EXPLAIN 명령어의 query_plan은 실제 쿼리문의 수행 시간을 저장하는 것이 아니라 추정된 시간을 저장하게 된다. 쿼리문을 실제로 수행하는데 고려하는 시간을 측정하려면 EXPLAIN 명령어에 ANALYZE 옵션을 추가하면된다. 

```Python
import psycopg2
import json
conn = psycopg2.connect(dbname="hud", user="hud_admin", password="hud123")
cur = conn.cursor()

cur.execute("EXPLAIN (ANALYZE, FORMAT json) SELECT COUNT(*) FROM homeless_by_coc WHERE year > '2012-01-01';")
query_plan = cur.fetchone()
print(json.dumps(query_plan, indent = 2))
```

ANALYZE 옵션은 Seq Scan을 측정하는 것 뿐만 아니라 일반적인 쿼리문을 수행하는데 걸리는 총 시간을 측정한다. DELETE, UPDATE, ALTER와 같은 DDL의 수행시간을 측정하려면 conn.rollback()을 추가로 작성해서 해당 쿼리가 실제 데이터베이스에 반영되지 않도록 한다. 

```Python
import psycopg2
import json
conn = psycopg2.connect(dbname="hud", user="hud_admin", password="hud123")
cur = conn.cursor()

cur.execute("EXPLAIN (ANALYZE, FORMAT json) DELETE FROM state_household_incomes;")
query_plan = cur.fetchone()

conn.rollback()
print(json.dumps(query_plan, indent=2))
```

# Using an Index

## Index Scan

쿼리문을 기본키(Primary Key)를 사용해서 작성하면 이번에 EXPLAIN 명령어를 사용했을 때 생성되는 "Seq Scan" 노드 타입이 아닌 "Index Scan"의 새로운 노드 타입이 생성되어있는 것을 확인할 수 있다. 테이블이 생성될 때 기본키를 설정하면 Postgres는 B-tree index를 생성해서 효율적인 쿼리를 할 수 있는 컬럼을 제공한다. 즉, "Seq Scan"방식은 정보를 찾기 위해 책의 모든 페이지를 다 읽는 방식이고, Index Scan 방식은 목차를 통해 원하는 정보로 이동하여 읽는 방식이다. 

```Python
import psycopg2
import json
conn = psycopg2.connect(dbname="hud", user="hud_admin", password="hud123")
cur = conn.cursor() 

cur.execute("""
    EXPLAIN (ANALYZE, FORMAT json) 
    SELECT *
        FROM homeless_by_coc
    WHERE id = 10;
""")
query_plan = cur.fetchone()
print(json.dumps(query_plan, indent=2))
```

## Comparision of time between Seq Scan and Index Scan 

측정결과 coc_name_plan의 실행시간은 0.746, id_plan의 실행시간은 0.097로 Index Scan의 수행시간이 압도적으로 빠름을 확인할 수 있다. 대체적으로 Index Scan 방식은 전체 테이블의 행에 대해 $log_{2}(N)$의 시간비용을 가지고 Seq Scan 방식은 $N$의 시간비용을 가지고 있다. 

```Python
import psycopg2
conn = psycopg2.connect(dbname="hud", user="hud_admin", password="hud123")
cur = conn.cursor() 

cur.execute("""
    EXPLAIN (ANALYZE, FORMAT json) 
    SELECT * 
        FROM homeless_by_coc
    WHERE coc_name = 'Chester County CoC'
    LIMIT 1;
""")
coc_name_plan = cur.fetchone() 
print(coc_name_plan[0][0]["Execution Time"])

cur.execute("""
    EXPLAIN (ANALYZE, FORMAT json) 
    SELECT * 
        FROM homeless_by_coc
    WHERE id=42704;
""")
id_plan = cur.fetchone()
print(id_plan[0][0]["Execution Time"])
```

## Create Index

하지만 데이터를 찾는 과정에서 기본키를 사용하는 경우는 매우 드물다. 따라서 기본키를 사용하지 않더라도 Index Scan을 사용하는 방법이 필요하게 된다. Postgres는 INDEX라는 객체를 사용해 Index Scan이 가능하도록 제고앟고 있다. 인덱스를 지정할때 해당 이름은 전체 데이터베이스에 똑같은 이름이 반드시 존재하면 안된다. 

하지만 Index는 각각의 인덱스를 저장할 저장 공간이 필요하기 때문에 공간상의 낭비를 불러오는 문제가 존재한다. 

```Python
import psycopg2
conn = psycopg2.connect(dbname="hud", user="hud_admin", password="hud123")
cur = conn.cursor() 

cur.execute("CREATE INDEX coc_name_index ON homeless_by_coc(coc_name);")
conn.commit()
conn.close()
```

## Information in pg_indexes

비록 Index Scan 방식이 Seq Scan보다 빠르긴 하지만 데이터베이스 내부에 인덱스가 저장되어야 하기때문에 데이터 공간을 필요로 하게 된다. 만약 너무 많은 공간이 필요하게 되면 Index Scan의 수행 속도는 Seq Scan과 거의 차이가 없어지게된다. 생성된 인덱스의 정보는 system catalog의 pg_indexes 테이블에 저장되어 있다. 기본적으로 인덱스가 생성되면 Postgres는 btree 인덱스를 생성한다. Postgres는 btree 인덱스 이외에 hash, gist, spgist, gin, brin을 제공하며 default로 btree를 사용한다. hash 인덱스의 경우 WHERE 절에서 조건을 제시할때 사용된다.

| Name | Type | References | Description | 
|:---:|:---:|:---:|:---:|
|schemaname|name|pg_namespace.nspname||
|tablename|name|pg_class.relname||
|indexname|name|pg_class.relname||
|tablespace|name|pg_tablespace.spcname||
|indexdef|text||Index dfinition (a reconstructed CREATE INDEX command)|

- schemaname, tablename : 인덱스가 생성된 스키마와 테이블의 이름
- indexdef : 인덱스가 생성된 방식(기본키의 경우 CREATE UNIQUE INDEX, 생성된 인덱스의 경우 CREATE INDEX로 표기)


## Drop index 

생성한 인덱스가 차지하는 공간이 매우 크다면 Index Scan을 사용하는 방식은 Seq Scan을 사용하는 방식과 차이가 없다. 따라서 Index를 사용하기 위해 DROP INDEX문을 사용한다. 하지만 매치되는 인덱스 이름이 없을경우 오류를 출력하는데, 이를 생략하기 위해 IF EXISTS를 사용해 오류가 출력되는 것을 방지할 수 있다.

```Python
import psycopg2
import json
conn = psycopg2.connect(dbname="hud", user="hud_admin", password="hud123")
cur = conn.cursor() 

cur.execute("DROP INDEX IF EXISTS coc_name_index;")
conn.commit()
conn.close()
```

## Workflow of appplying index scan 

```Python
import psycopg2
import json
conn = psycopg2.connect(dbname="hud", user="hud_admin", password="hud123")
cur = conn.cursor() 

cur.execute("""
    EXPLAIN (ANALYZE, FORMAT json)
    SELECT homeless_by_coc.state, homeless_by_coc.coc_number, homeless_by_coc.coc_name, state_info.name 
        FROM homeless_by_coc, state_info
    WHERE homeless_by_coc.state = state_info.postal;
""")
no_index_plan = cur.fetchone()
print(no_index_plan[0][0]["Execution Time"])

cur.execute("CREATE INDEX state_index ON homeless_by_coc(state);")

cur.execute("""
    EXPLAIN (ANALYZE, FORMAT json)
    SELECT homeless_by_coc.state, homeless_by_coc.coc_number, homeless_by_coc.coc_name, state_info.name 
        FROM homeless_by_coc, state_info
    WHERE homeless_by_coc.state = state_info.postal;
""")
index_plan = cur.fetchone()
print(index_plan[0][0]["Execution Time"]) 
conn.commit()
```

해당 코드의 수행결과 각각 58.063과 58.443으로 큰 차이가 존재하지 않음을 확인할 수 있다. 왜냐하면 인덱스를 적용하더라도 INNER JOIN ~ ON ~ 절 내부에서 확인해야하는 값이 전체 테이블의 행의 개수와 큰 차이가 없기 때문에 Seq Scan을 하는 것과 마찬가지이기 때문이다. 하지만 WHERE절과 같은 특수한 조건을 사용해서 행을 제한하는 경우 Index Scan이 압도적으로 효율적이다.

```Python
import psycopg2
import json
conn = psycopg2.connect(dbname="hud", user="hud_admin", password="hud123")
cur = conn.cursor() 

cur.execute("""
    EXPLAIN (ANALYZE, FORMAT json)
    SELECT homeless_by_coc.state, homeless_by_coc.coc_number, homeless_by_coc.coc_name,     state_info.name
        FROM homeless_by_coc
    INNER JOIN state_info
    ON homeless_by_coc.state = state_info.postal
    WHERE homeless_by_coc.count > 5000;
""")
no_index_plan = cur.fetchone()
print(no_index_plan[0][0]["Execution Time"])

cur.execute("CREATE INDEX count_index ON homeless_by_coc(count);")

cur.execute("""
    EXPLAIN (ANALYZE, FORMAT json)
    SELECT homeless_by_coc.state, homeless_by_coc.coc_number, homeless_by_coc.coc_name,     state_info.name
        FROM homeless_by_coc
    INNER JOIN state_info
    ON homeless_by_coc.state = state_info.postal
    WHERE homeless_by_coc.count > 5000;
""")
index_plan = cur.fetchone()
print(index_plan[0][0]["Execution Time"])

conn.commit()
```

# Advanced Index

## Bitmap Index and Heap Scan 

Index Scan은 Seq Scan에 비해 속도면에서 훨씬 빠르다. SQL에는 WHERE문에 단일 조건문이 아니라 복합 조건문을 흔히 사용하게 된다. 이때 한 컬럼은 Index로 나머지 컬럼은 Index로 지정이 되지 않았다면 query_plan의 시행결과는 Bitmap Index Scan과 Bitmap Heap Scan의 nested structure가 형성되게 된다.

```Python
import json
import psycopg2
conn = psycopg2.connect("dbname=hud user=hud_admin password=hud_pwd")
cur = conn.cursor()

cur.execute("CREATE INDEX state_index ON homeless_by_coc(state);")
conn.commit()
cur.execute("""
    EXPLAIN (FORMAT json) 
    SELECT * 
        FROM homeless_by_coc
    WHERE state = 'CA' AND year < '2008-01-01';
""")
query_plan = cur.fetchone()
print(json.dumps(query_plan, indent=2))

# Result of query_plan in josn 
[
  {
    "Plan": {
      "Node Type": "Bitmap Heap Scan",
      "Parallel Aware": false,
      "Relation Name": "homeless_by_coc",
      "Alias": "homeless_by_coc",
      "Startup Cost": 11.7,
      "Total Cost": 903.46,
      "Plan Rows": 144,
      "Plan Width": 12,
      "Recheck Cond": "(state = 'CA'::bpchar)",
      "Filter": "(year < '2008-01-01'::date)",
      "Plans": [
        {
          "Node Type": "Bitmap Index Scan",
          "Parent Relationship": "Outer",
          "Parallel Aware": false,
          "Index Name": "state_index",
          "Startup Cost": 0.0,
          "Total Cost": 11.67,
          "Plan Rows": 433,
          "Plan Width": 0,
          "Index Cond": "(state = 'CA'::bpchar)"
        }
      ]
    }
  }
]
```

## How Postgres Stores its data

Postgres 내부 테이블의 컬럼은 heap page라는 집합으로 분리되어 있다. Postgres 내부에서 인덱스를 생성하면 Postgres는 쿼리문에서 요구하는 행이 존재하는 page를 찾게 된다. 즉, b-tree 구조를 사용한 Index Scan 방식은 Postgres가 b-tree에게 heap page와 page 행의 위치를 질문하고 행을 검색하게 된다. 

해당 방식은 각각의 행에 따른 heap page를 로드해야하기 때문에 page loading cost가 발생한다. 예를들어 Index Scan을 사용하면 page1, page2, page3, page3을 각각 로드하게 된다.

대안적인 방법으로 Index Scan 방식을 사용해서 행이 존재하는 heap page를 찾고, 해당 page 전체를 seq scan하는 방식은 page1, page2, page3으로 중복되는 page loading없이 행을 검색하는 절충적인 방법이다. 즉, Bitmap Index Scan은 결과를 만족하는 heap page를 찾을 뿐 어떤 행인지 파악하지 못하고, Bitmap Heap Scan에서 Recheck Cond 필드에 존재하는 조건을 통해 Seq Scan하는 방식으로 조건을 만족하는 행을 찾게 된다.

## Multi-Column index 

두개의 컬럼을 동시에 인덱스로 설정해서 사용하는 방식을 **multi-column index**라고 한다. multi-column index를 설정하는 방식은 단일 인덱스를 생성하는 방법과 동일하며 복수의 컬럼을 ','로 분리해 기입한다. 

```Python
import psycopg2
import json
conn = psycopg2.connect("dbname=hud user=hud_admin password=hud_pwd")
cur = conn.cursor()

cur.execute("""
    EXPLAIN (ANALYZE, FORMAT json) 
    SELECT * 
        FROM homeless_by_coc
    WHERE state = 'CA' AND year < '2008-01-01';
""")
plan_single_index = cur.fetchone() 
print(plan_single_index[0][0]["Execution Time"])

cur.execute("CREATE INDEX state_year_index ON homeless_by_coc(state, year);")
conn.commit() 

cur.execute("""
    EXPLAIN (ANALYZE, FORMAT json) 
    SELECT * 
        FROM homeless_by_coc
    WHERE state = 'CA' AND year < '2008-01-01';
""")
plan_multi_index = cur.fetchone()
print(plan_multi_index[0][0]["Execution Time"])
```

## Cautions of Multi-Column index 

Multi-Column index를 테이블 전체의 칼럼에 대해서도 적용할수 있지만, 모든 행을 인덱스로 정의하더라도 시간면에서 크게 도움이 되지 않을 수 있다. Multi-Column index를 정의할 때 입력된 컬럼의 순서로 테이블이 정렬되기 때문에, 적절하지 않은 순서로 정렬된 테이블은 오히려 Seq Scan보다 비효율적인 쿼리문을 작동하게 한다. 

예를들어 state-count-year 순으로 정렬된 테이블은 year, count 순서로 데이터를 검색하게 될 경우 state에 따라 먼저 정의된 테이블이 존재하기 때문에 Index Scan에 대해 비효율적이다.

```Python
import psycopg2
import json
conn = psycopg2.connect("dbname=hud user=hud_admin password=hud_pwd")
cur = conn.cursor()

cur.execute("CREATE INDEX state_count_year_index ON homeless_by_coc(state, count, year);")
conn.commit() 

cur.execute("""
    EXPLAIN (ANALYZE, FORMAT json) 
    SELECT * 
        FROM homeless_by_coc
    WHERE year > '2011-01-01' AND count > 5000;
""")
query_plan = cur.fetchone()
print(json.dumps(query_plan, indent=2))

# Result of query_plan in json 
[
  [
    {
      "Plan": {
        "Node Type": "Seq Scan",
        "Parallel Aware": false,
        "Relation Name": "homeless_by_coc",
        "Alias": "homeless_by_coc",
        "Startup Cost": 0.0,
        "Total Cost": 2579.95,
        "Plan Rows": 9614,
        "Plan Width": 480,
        "Actual Startup Time": 3.25,
        "Actual Total Time": 11.208,
        "Actual Rows": 251,
        "Actual Loops": 1,
        "Filter": "((year > '2011-01-01'::date) AND (count > 5000))",
        "Rows Removed by Filter": 86279
      },
      "Planning Time": 0.342,
      "Triggers": [],
      "Execution Time": 11.259
    }
  ]
]
```

## Index after applying Built-in functions 

문자열 데이터를 검색할 때 사용하는 가장 흔한 방식은 cas-insensitive 이다. 따라서 WHERE문에 LOWER() 혹은 UPPER()같은 함수를 적용하게 된다. 하지만 내장 함수를 컬럼에 적용하면 저장된 인덱스를 고려하지 않고 조건문을 사용해 Seq Scan을 하게 된다. 따라서 인덱스를 적용할 때 적용된 내장함수를 함께 사용하는 방식으로 인덱스를 생성해야 한다.

```Python
import psycopg2
conn = psycopg2.connect("dbname=hud user=hud_admin password=hud_pwd")
cur = conn.cursor()

cur.execute("CREATE INDEX measures_index ON homeless_by_coc(LOWER(measures));")
conn.commit() 

cur.execute("""
    SELECT * 
        FROM homeless_by_coc
    WHERE LOWER(measures) = 'total homeless';
""")
total_home
```

## Partial index 

또한 조건에 따라 행의 일부분만을 인덱스로 지정할 수 있다. **partial index**는 인덱스로 저징한 행뿐만 아니라 다른 컬럼을 기준으로 인덱스의 부분을 지정할 수 있다. partial index를 사용하는 이유는 빈번하게 발생하는 값의 인덱싱을 피하기 위해서이다. 행의 길이가 매우 길어지면 Index Scan을 사용하지 않고 Seq Scan을 사용하게 되는데, 인덱스를 제한하면 인덱스를 저장하는데 필요한 공간을 감소시킬 뿐만 아니라 쿼리문을 가속하는데 사용할 수 있다. 

```Python
import psycopg2
conn = psycopg2.connect("dbname=hud user=hud_admin password=hud_pwd")
cur = conn.cursor()

cur.execute("CREATE INDEX partial_state_index ON homeless_by_coc(state) WHERE count = 0;")
conn.commit()
cur.execute("""
    SELECT *
        FROM homeless_by_coc
    WHERE state='CA' AND count = 0;
""")
ca_zero_count = cur.fetchall()
```


## Workflow of applying proper index 

```Python
import psycopg2
conn = psycopg2.connect("dbname=hud user=hud_admin password=hud_pwd")
cur = conn.cursor()

cur.execute("""
    CREATE INDEX state_measure_index 
    ON homeless_by_coc(state, LOWER(measures)) 
    WHERE count = 0
""")
conn.commit() 
cur.execute("""
    EXPLAIN (ANALYZE, FORMAT json) 
    SELECT hbc.year, si.name, hbc.count
    FROM homeless_by_coc AS hbc
    INNER JOIN state_info AS si
    ON hbc.state = si.postal
    WHERE hbc.state = 'CA' AND hbc.count = 0 AND LOWER(hbc.measures) = 'total homeless';
""")
print(json.dumps(cur.fetchone(), indent=2))

# Result of quey_plan in json 
[
  [
    {
      "Plan": {
        "Node Type": "Nested Loop",
        "Parallel Aware": false,
        "Join Type": "Inner",
        "Startup Cost": 0.41,
        "Total Cost": 24.84,
        "Plan Rows": 3,
        "Plan Width": 54,
        "Actual Startup Time": 0.041,
        "Actual Total Time": 0.042,
        "Actual Rows": 0,
        "Actual Loops": 1,
        "Inner Unique": false,
        "Plans": [
          {
            "Node Type": "Index Scan",
            "Parent Relationship": "Outer",
            "Parallel Aware": false,
            "Scan Direction": "Forward",
            "Index Name": "state_measure_index",
            "Relation Name": "homeless_by_coc",
            "Alias": "hbc",
            "Startup Cost": 0.41,
            "Total Cost": 8.43,
            "Plan Rows": 1,
            "Plan Width": 20,
            "Actual Startup Time": 0.04,
            "Actual Total Time": 0.04,
            "Actual Rows": 0,
            "Actual Loops": 1,
            "Index Cond": "((state = 'CA'::bpchar) AND (lower((measures)::text) = 'total homeless'::text))",
            "Rows Removed by Index Recheck": 0
          },
          {
            "Node Type": "Seq Scan",
            "Parent Relationship": "Inner",
            "Parallel Aware": false,
            "Relation Name": "state_info",
            "Alias": "si",
            "Startup Cost": 0.0,
            "Total Cost": 16.38,
            "Plan Rows": 3,
            "Plan Width": 58,
            "Actual Startup Time": 0.0,
            "Actual Total Time": 0.0,
            "Actual Rows": 0,
            "Actual Loops": 0,
            "Filter": "(postal = 'CA'::bpchar)",
            "Rows Removed by Filter": 0
          }
        ]
      },
      "Planning Time": 0.594,
      "Triggers": [],
      "Execution Time": 0.073
    }
  ]
]
```

# Vacumming Postgres Database 

## Speed Problem after deleting all rows in table 

대표적인 destructvie queries 중 하나인 UPDATE와 DELETE 명령어의 경우, 해당 명령어를 사용하고 테이블에 다시 데이터를 업로드하면 데이터의 내용은 변경된 것이 없지만 업로드 속도가 이전에 비해 느려진 것을 확인할 수 있다.

```python
import psycopg2
conn = psycopg2.connect(dbname="hud", user="hud_admin", password="hud123")
cur = conn.cursor() 

cur.execute("SELECT * FROM homeless_by_coc;")
num_rows_before = len(cur.fetchall())

cur.execute("DELETE FROM homeless_by_coc;")
cur.execute("SELECT * FROM homeless_by_coc;")
num_rows_after = len(cur.fetchall())

print(num_rows_before, num_rows_after)
```

DELETE 명령어를 사용하게되면 기존에 존재하는 데이터 테이블 내부의 행은 **dead rows** 상태로 존재하게 된다. 이는 Window 운영체제에서 파일을 삭제하더라도 '휴지통' 폴더 내부에는 파일이 아직 남아있는 것과 같다. Postgres는 서로 다른 사용자가 데이터베이스 서버에 접속해서 사용하기 때문에 사용자들의 데이터의 **일관성(Consistency)**를 보장할 필요가 있다. 

Postgres는 서로 다른 유저들의 일관성을 보장하기위해 multi-version control을 사용한다.


## Multi-version concurrency control(MVCC)

**다중 버전 동시성 제어(multiversion concurrency control)**은 데이터베이스 관리 시스템이 일반적으로 사용하는 동시성 제어 방식으로, 데이터베이스로의 동시 접근을 제공하고 프로그래밍 언어에서 트랜잭셔널 메로리를 구현한다.

동시성 제어 없이 누군가가 데이터베이스를 읽는 동시에 다른 누군가가 기록을 하려고 한다면 독자는 반쯤 기록되거나 일관성이 없는 데이터를 보게 될 가능성이 있다. 

MVCC 데이터베이스가 데이터의 업데이트가 필요할 때, 기존 데이터 항목을 새로운 데이터가 덮어쓰는 대신 데이터 항목의 새로운 버전을 만든다. 즉, 여러 버전이 저장된다. 

## Check number of dead rows in table 

dead rows는 트랜잭션 내부의 데이터베이스 테이블에 대한 응집성을 보장하지만 테이블에 SELECT 문을 적용하게 되면 결국 전체 테이블의 사이즈가 증가하였기 때문에 수행시간이 중가하게 된다. 테이블의 dead rows의 수에 대한 정보는 pg_catalog 스키마의 pg_stat_all_tables 테이블을 확인한다.

```Python
import psycopg2
conn = psycopg2.connect(dbname="hud", user="hud_admin", password="hud123")
cur = conn.cursor()

cur.execute("SELECT n_dead_tup FROm pg_catalog.pg_stat_all_tables WHERE relname = 'homeless_by_coc';")
homeless_dead_rows = cur.fetchone()[0]
print(homeless_dead_rows)
```

## Vacuum dead rows in table 

VACUUM 명령어는 테이블 내부에 존재하는 dead rows를 제거한다. 만약 테이블명 옵션없이 사용하면 모든 유저, 테이블을 처리하게 된다. VACUUM 명령어의 주요 목표는 reclaim table stoage space occupied by dead tuples로, 테이블을 차지하고 있는 dead rows를 제거하는 것이다.

VACUUM 명령어는 트랜잭션 블록 내부에서 쓰이면 안되기 때문에 connection.autocommit = True를 설정해야 한다. 또한 VERBOSE 옵션은 제거된 dead rows의 개수를 반환하여 connection.notices 속성 내부에 저장한다. 

```Python
import psycopg2
conn = psycopg2.connect(dbname="hud", user="hud_admin", password="hud123")
conn.autocommit = True 
cur = conn.cursor()

cur.execute("VACUUM VERBOSE homeless_by_coc;")
for notice in conn.notices : 
    print(notice)
```

```Python
INFO:  vacuuming "public.homeless_by_coc"

INFO:  scanned index "homeless_by_coc_pkey" to remove 86530 row versions
DETAIL:  CPU: user: 0.00 s, system: 0.00 s, elapsed: 0.00 s

INFO:  "homeless_by_coc": removed 86530 row versions in 1280 pages
DETAIL:  CPU: user: 0.00 s, system: 0.00 s, elapsed: 0.00 s

INFO:  index "homeless_by_coc_pkey" now contains 0 row versions in 239 pages
DETAIL:  86530 index row versions were removed.
236 index pages have been deleted, 0 are currently reusable.
CPU: user: 0.00 s, system: 0.00 s, elapsed: 0.00 s.

INFO:  "homeless_by_coc": found 86530 removable, 0 nonremovable row versions in 1280 out of 1280 pages
DETAIL:  0 dead row versions cannot be removed yet, oldest xmin: 534
There were 0 unused item identifiers.
Skipped 0 pages due to buffer pins, 0 frozen pages.
0 pages are entirely empty.
CPU: user: 0.01 s, system: 0.00 s, elapsed: 0.02 s.

INFO:  "homeless_by_coc": truncated 1280 to 0 pages
DETAIL:  CPU: user: 0.00 s, system: 0.00 s, elapsed: 0.00 s
```

- nonremovable row versions : dead rows가 제거되기 전 다른 트랜잭션이 시작되었을 경우 ACID를 보장하기 위해 남아있는 행의 수(혹은 DELETE 되지 않는 실제 행) 
- xmin : 행을 삽입하는 트랜잭션의 ID 
- xmax : 행을 제거하는 트랜잭션의 ID 

Postgres의 xmin, xmax는 실제 수행되고 있는 트랜잭션이 존재하는지 알기위해 사용하며 행을 영구적으로 삭제할지 결정하는데 도움을 준다. 

```Python
import psycopg2
row = (1, '2007-01-01', 'AK', 'AK-500', 'Anchorage CoC', 'Chronically Homeless Individuals', 224)

conn = psycopg2.connect(dbname="hud", user="hud_admin", password="hud123")
conn.autocommit = True 
cur = conn.cursor() 

cur.execute("INSERT INTO homeless_by_coc VALUES (%s, %s, %s, %s, %s, %s, %s);", row)
cur.execute("VACUUM VERBOSE homeless_by_coc;")
for notice in conn.notices : 
    print(notice) 

cur.execute("SELECT xmin FROM homeless_by_coc;")
xmin = cur.fetchone() 
print(xmin)
```

## Advantages of using Vacuum

### ANALYZE option

VACUUM 명령어의 장점은 테이블 통계량를 업데이트하는 능력이다. EXPLAIN 명령어의 주된 약점은 테이블 통계 데이터의 부정확성이었다. 하지만 VACUUM명령어의 ANALYZE옵션은 pg_stats에 새로운 비용과 행의 개수를 업데이트해 정확성을 보강한다.

```Python
import json
conn = psycopg2.connect(dbname="hud", user="hud_admin", password="hud123")
conn.autocommit = True
cur = conn.cursor() 

cur.execute("EXPLAIN SELECT * FROM homeless_by_coc;")
plan_before = cur.fetchall() 

cur.execute("VACUUM ANALYZE homeless_by_coc;") 

cur.execute("EXPLAIN SELECT * FROM homeless_by_coc;")
plan_after = cur.fetchall()

print(plan_before)
print()
print(plan_after)
```

### FULL option

테이블을 vacuum 하면, freeze된 디스크 공간(dead rows가 차지하고 있던 공간)은 그대로 남아있어 새로운 데이터가 할당되는데 사용된다. FULL 옵션은 전체 데이터베이스 서버에 대해 나중에 사용할 수 있도록 공간을 비우게 된다. 이 작업은 전체 데이터베이스 서버에 대해 공간을 재정의하지만, vacuumming 하는 동안엔 exclusive lock을 선언하여 어떠한 쿼리문도 도중에 수행할 수 없게 된다. 

pg_total_relation_size 함수는 테이블이 사용하는 디스크 용량을 의미하고, pg_size_pretty 함수는 pg_total_relation_size 함수에서 나온 결과를 읽기 쉬운 값으로 변환해준다. 

```Python
import psycopg2
conn = psycopg2.connect(dbname="hud", user="hud_admin", password="hud123")
conn.autocommit = True
cur = conn.cursor() 

cur.execute("SELECT * FROM homeless_by_coc;")
cur.fetchall() 

cur.execute("SELECT pg_size_pretty(pg_total_relation_size('homeless_by_coc'));")
space_before = cur.fetchone()

cur.execute("VACUUM FULL homeless_by_coc;")
cur.execute("SELECT pg_size_pretty(pg_total_relation_size('homeless_by_coc'));")
space_after = cur.fetchone()

print(space_before, space_after)
```

## Autovacuum 

Postgres는 테이블에 대해 자동적으로 VACUUM 명령어를 적용할 수 있는 기능을 제공한다. 해당 특징은 autovacuum으로 테이블에 주기적으로 적용되어 dead rows를 제거하고 통계량이 최신화되도록 보장한다. 다행히 최신 버전의 Postgres는 autovacuum을 자동으로 적용하도록 업데이트 되어있다.

VACUUM을 적용하는 기준은 다음과 같다.

- 많은 양의 새로운 행을 추가하거나 주 테이블의 행을 삭제하는 것 없이 일반적인 분석 업무를 진행하고 있는가?
- 테이블 내부의 많은 양의 데이터를 최근에 삭제하고 복잡한 명령을 수행하고자 하는가?
- 데이터 서버에 free 공간이 부족하는가?

이러한 질문들은 vacuum 명령어를 적용할 것인지를 알려주는 가이드라인으로 제공된다. 즉, 쿼리문을 수행하는데 성능 향상을 위한 방법으로 EXPLAIN 명령어로 쿼리의 노드타입을 확인하고, n_dead_tup 컬럼에서 dead rows의 수를 확인한 작업을 수행하는 작업은 VACUUM 명령어를 적용하기 위한 기준이 된다.

VACUUM이 언제 되었는지 확인하는 테이블은 pg_stat_user_tables 의 last_vacuum과 last_autovacuum 컬럼에서 확인 가능하다.

```Python
import psycopg2
conn = psycopg2.connect(dbname="hud", user="hud_admin", password="hud123")
conn.autocommit = True
cur = conn.cursor()

cur.execute("VACUUM homeless_by_coc;")
import time 
time.sleep(1)

cur.execute("SELECT last_vacuum, last_autovacuum FROM pg_stat_user_tables WHERE relname = 'homeless_by_coc';")
timestamps = cur.fetchone()
print(timestamps)
```